In [ ]:
from selenium import webdriver

import pandas as pd
import json
import os

# read the file with the program information
Programas = pd.read_csv('programs.csv')
# set the settings for Chromium
browser = webdriver.Firefox()
# configure to save the page as pdf
options = webdriver.Firefox()
options.add_argument("--kiosk-printing")


settings = {
    "recentDestinations": [{
        "id": "Save as PDF",
        "origin": "local",
        "account": "",
        
    }],
    "selectedDestinationId": "Save as PDF",
    "version": 2,

}
# define  the path in which the pdf file will be save, in this case the folder /home/user/Selenium/pdf_files_temp which is a temporal folder, because after save it will be necessary change file name 
prefs = {'printing.print_preview_sticky_settings.appState': json.dumps(settings), "savefile.default_directory": r"/home/jc/pdf_files_temp",}
options.add_experimental_option('prefs', prefs)

# This loop will be repeated the number of lines of the programs.csv file
for x in range(1,len(Programas)):
    # create the string to be search on google. 
    String = Programas['ProgramName'][x] + "+" + Programas['UniversityName'][x] + "+ perfil del egresado"
    # open the driver (the web browser) in this case Firefox

    driver = webdriver.Firefox(options=options)
    # define the url adding the string 
    url = "https://www.google.com/search?client=firefox-b-d&q=" + String
    # load the url on the web browser
    driver.get(url)
    # Path2 is the "name" of the first result on google search
    Path2="//div[@class='yuRUbf']/a[@href]"
    # click on the first google result
    driver.find_element("xpath", Path2).click()
    # In the next try-except blocks the sript try to identify if on the post graduation program website appears the words "O programa" ou "Objetivo". When some of these words appear on the website, the script save the page as pdf, otherwise pass to the next csv line. 
    try:
        driver.find_element("xpath", "//*[contains(text(), 'O programa')]")
        driver.execute_script('window.print();')
    except: 
        print("Apresentação not found")
    
    try:
        driver.find_element("xpath", "//*[contains(text(), 'o programa')]")
        driver.execute_script('window.print();')
    except: 
        print("o programa not found")
        
    try:
        driver.find_element("xpath", "//*[contains(text(), 'objetivo')]")
        driver.execute_script('window.print();')
    except: 
        print("Objetivo not found")
    try:
        driver.find_element("xpath", "//*[contains(text(), 'OBJETIVO')]")
        driver.execute_script('window.print();')
    except: 
        print("OBJETIVO not found")
        
    driver.quit()
    try:
        ## Change the file name using the university and program name
        # Ones the file was saved, the script read the files on the folder path
        archivo = os.listdir('/home/user/Selenium/pdf_files_temp')
        # From all the files on the temporary folder, we choose the first
        archivo2 = os.path.join("/home/user/Selenium/pdf_files_temp/", archivo[0])
        # We rename the file name using the university and program name
        Nombre = os.path.join("/home/user/Selenium/pdf_files/" + Programas['NM_PROGRAMA_IES'][x] + "_" + Programas['NM_ENTIDADE_ENSINO'][x] + ".pdf") 
        # move the original file from the temporal folder to the results folder changing its name
        os.rename(archivo2, Nombre)
    except:
        print("File not found")
    
    # after rename, the rest of the file on the temporal folder will be erased   
    path = r"/home/user/Selenium/pdf_files_temp/"
    for file_name in os.listdir(path):
   
        file = path + file_name
        if os.path.isfile(file):
            print('Deleting file:', file)
            os.remove(file)
    # showing progress
    print('Trial',{x} )